# Chapter 9 - Unsupervised Learning Techniques
The vast majority of the available data is actually unlabeled. <br>
Clustering: the goal is to group similar instances together into clusters. <br>
Anomaly detection: the objective is to learn what "normal" data looks like, and use this to detect abnormal instances. <br>
Density estimation: this is the task of estimating the probability density function of the random process that generated the dataset.

## Clustering
Cluster: a group of similar instances.

Aplications of clustering:
- Customer segmentation
- Data analysis
- Dimensionality reduction technique: each instance's feature vector x can be replaced with the vector of its affinities (*affinity* is any meausre of how well an instance fits into a cluster).
- Anomaly detection
- Semi-supervised learning
- Search engines
- Segmenting an image

### K-Means
The K-Means algorithm is a simple algorithm capable of clustering blobs of instances very quickly and efficiently.

In [ ]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

In the context of clustering, an instances's label is the index of the cluster that this instance gets assigned to by the algorithm. <br>
The KMeans instance preserves a copy of the labels of the instances it was tained on available via the `labels_` instance variable:

In [ ]:
y_pred
# array([4, 0, 1, ..., 2, 1, 0], dtype=int32)

y_pred is kmeans.labels_
# True

kmeans.cluster_centers_
# array([[-2.80389616, 1.80117999],
#        [ 0.20876306, 2.25551336],
#        [-2.79290307, 2.79641063],
#        [-1.46679593, 2.28585348],
#        [-2.80037642, 1.30082566]])

X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.predict(X_new)
# array([1, 1, 2, 2], dtype=int32)

The K-Means algorithm does not behave very well when blobs have very different diamters since it cares the distance to the centroid the most.

Hard clustering: assigning each instance to a single cluster. <br>
Soft clustering: giving instance a score per cluster.

In [ ]:
kmeans.transform(X_new) #measuring the distance from each instance to every centroid
# array([[2.81093633, 0.32995317, 2.9042344 , 1.49439034, 2.88633901],
#        [5.80730058, 2.80290755, 5.84739223, 4.4759332 , 5.84236351],
#        [1.21475352, 3.29399768, 0.29040966, 1.69136631, 1.71086031],
#        [0.72581411, 3.21806371, 0.36159148, 1.54808703, 1.21567622]])

#### The K-Means Algorithm
After locating the centroids randomly, the algorithm repeats labeling the instances and updating the centroids until the centroids stop moving.

#### Centroid Initialization Methods
If you know approximately where the centroids shoud be, set the `init` hyperparameter to a NumPy array containing the list of centroids, and set `n_init` to 1.

In [ ]:
good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1)

Or you can run the algorithm multiple times (this is controlled by `n_init`) with different random initializations and keep the best soluthion with the lowest inertia. <br>
*Inertia*: the mean squared distance between each instance and its closest centroid.

In [ ]:
kmeans.inertia_
# 211.59853725816856

kmeans.score(X)
# -211.59853725816856

*K-Means*+\\+: a smater initialization step selects centroids that are distant from one another, and this makes the K-Means algorithm much less likely to converge to a sub-optimal solution. <br>
The `KMeans` class uses this initialization method by default.

#### Accelerated K-Means and Mini-batch K-Means
Accelerated K-Means: it accelerates the algorithm by avoiding many unnecessary distance calculations which is achieved by exploiting the trangle inequalities and by keeping track of lower and upper bounds for distances between instances and centroids. It is default by the `KMeans` class. 

Mini-batch K-Means: the algorithm is capable of using mini-batches, moving the centroids just slightly at each iteration instead of using the full dataset at each iteration. <br>
Although the Mini-batch K-Means algorithm is much faster than the regular K-Means algorithm, its inertia is generally slightly worse, especially as the number of clusters increases.

In [ ]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

#### Finding the Optimal Number of Clusters
*Silhouette score*: the mean silhouette coefficient over all the instances. <br>
*Silhouette coefficient*: $(b - a) / max(a, b)$ <br>
$a$: the mean distance to the other instances in the same cluster. <br>
$b$: the mean distance to the instances of the next closest cluster. 

The silhouette coefficient can vary between -1 and +1: a coefficient close to +1 means that the instance is well inside its own cluster and far from other clusters, while a coefficient close to 0 means that it is close to a cluster boundary, and finally a coefficient close to -1 means that the instance may have been assigned to the wrong cluster.

*Silhouette diagram* & *Silouhette analysis*
When most of the instances in a cluster have alower coefficient than the silhouette score, then the cluster is rather bad since this means its instances are much too close to other clusters. <br>
Choose $k$ where all clusters have similiar sizes.

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)
# 0.655517642572828

### Limits of K-Means
- It is necessary to run the algorithm several times to avoid sub-optimal solutions.
- You need to specify the number of clusters.
- K-Means does not behave very well when the clusters have varying sizes, different densities, or non-spherical shapes.

It is important to scale the input features before you run K-Means, or else the clusters may be very stretched, and K-Means will perform poorly.

### Using clustering for image segmentation
*Image Segmentation*: the task of partitioning an image into multiple segments. <br>
In *semantic segmentation*, all pixels that are part of the same object type get assigned to the same segment. <br>
In *instance segmentation*, all pixels that are part of the same individual object are assigned to the same segment. <br>
*Color segmentation*: simply assign pixels to the same segment if they have a similar color.

In [ ]:
from matplotlib.image import imread
image = imread(os.path,join("images", "clustering", "ladybug.png"))
image.shape
# (533, 800, 3)    # the height, the width, and the number of color channels

In [ ]:
X = image.reshape(-1, 3) # It reshapes the array to get a long list of RGB colors
kmeans = KMeans(n_cluster=8).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape) 

### Using Clustering for Preprocessing


In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

X_digits, y_digits = load_digits(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits) 

pipeline = Pipeline([Pipeline
    ("kmeans", Kmeans(n_clusters=50)),
    ("log_reg", LogisticRegression())
])

pipeline.fit(X_train, y_train)
# 0.9822222222222222

In the previous code, we created a pipeline that first clustered the training set into 50 clusters and replaced the images with their distances to these 50 clusters, then apply a logistic regression model.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

grid_clf.best_params_
# {'kmeans__n_clusters: 90}
grid_clf.score(X_test, y_test)
# 0.9844444444444445

We used `GridSearchCV` to find the optimal number of clusters.

### Using Clustering for Semi-Supervised Learning


In [ ]:
k = 50
kmeans = KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

y_representative_digits = np.array([4, 8, 0, 6, 8, 3, ..., 7, 6, 2, 3, 1, 1])

log_reg = LogisticRegression()
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)
# 0.9244444444444444

First, we clustered the training set into 50 clusters, then for each cluster we find the representative images (images closest to the centroid. Next, we manually label each image). 

*Label propagation*: propagating the labels to all the other instances in the same cluster. 

In [ ]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test, y_test)
# 0.9288888888888889

Propagating the labels to the 20% of the instances that are closest to the centroids:

In [ ]:
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1\

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)
# 0.9422222222222222

np.mean(y_train_partially_propagated == y_train[partially_propagated])
# 0.9896907216494846

*Active learning*: a human expert interacts with the learning algorithm, prividing labels when the algorithm needs them. One of the most common ones is called uncertainty sampling.<br>

*Uncertainty sampling*:
- The model is trained on the labeld instances gathered so far, and this model is used to make predictions on all the unlabeled instances.
- The instances for which the model is most uncertain (i.e., when its estimated probability is lowest) must be labeled by the expert.
- Then you just iterate this process again and again, until the performance imporvement stop being worth the labeling effort. 

### DBSCAN
- For each instance, the algorithm counts how many instances are located within a small distance $\varepsilon$ from it. THis region is called the instance's $\varepsilon$-neighborhood.
- If an instance has at least `min_samples` instances in its $varepsilon$-neighborhood (including itself), then it is considered a *core instance*. (A dense region)
- All instances in the neighborhood of a core instance belong to the same cluster. This may include other core instances, therefore a long sequence of neighboring core instances forms a single cluster.
- Any instance that is not a core instance and does not have one in its neighborhood is considered an anomaly.

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

dbscan.labels_ # Anomaly if -1.
# array([ 0, 2, -1, -1, 1, 0, 0, 0, ..., 3, 2, 3, 3, 4, 2, 6, 3])

len(dbscan.core_sample_indices_)
# 808

dbscan.core_sample_indices_
# array([ 0, 4, 5, 6, 7, 8, 10, 11, ..., 992, 993, 995, 997, 998, 999])

dbscan.components_
# array([[-0.02137124, 0.40618608],
#        [-0.84192557, 0.53058695],
#        ...
#        [-0.94355873, 0.3278936 ],
#        [ 0.79419406, 0.60777171]])


The DBSCAN class does not have a `predict()` method, and it cannot predict which cluster a new instance belongs to. <br>
The rationale for this decision is that several classification algorithms could make sense here, and it is easy enough to train one.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

X_new = np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2, 1]])

knn.predict(X_new)
# array([1, 0, 1, 0])

knn.predict_proba(X_new)
# array([[0.18, 0.82],
#        [1. , 0. ],
#        [0.12, 0.88],
#        [1. , 0. ]])

DBSCAN is a very simple yet powerful algorithm, capable of identifying any number of clusters, of any shape, it is robust to outliers, and it has just two hyperparameters. <br>
However, if the density varies significantly across the clusters, it can be impossible for it to caputre all the clusters properly.

### Other Clustering Algorithms
- Agglomerative clustering: a hierarchy of clusters is built from the bottom up.
- Birch: this algorithm was designed specifically for very large datasets, and it can be faster than batch K-Means, with similar results, as long as the number of features is not too large (<20).
- Mean-shift: this algorithm starts by placing a circle centered on each instance, then for each circle it computes the mean of all the instances located within it, and it shifts the circle so that it is centered on the mean. Next, it iterates this mean-shift step until all the circles stop moving. All the instances whose circles have settled in the same palce are assigned to the same cluster.
- Affinity propagation: this algorithm uses a voting system, where instances vote for similar instances to be their representatives, and once the algorithm converges, each representative and its voter form a cluster.
- Spectral clustering: this algorithm takes a similarity matrix between the instances and creates a low-dimensional embedding from it, then it uses another clustering algorithm in this low-dimensional space.

## Gaussian Mixtures
A *Gaussian mixture model* (GMM) is a probabilistic model that assumes that the instances were generated from a mixture of several Gaussian distributions whose parameters are unknown. <br>
All the instances generated from a sigle Gaussian distribution form a cluster that typically looks like an ellipsoid. <br>
*Expectation Maximization* (EM) algorithm: it initializes the cluster parameters randomly, then it repeats assigning instances to clusters (the expectation step) and updating the clusters (the maximization step). <br>
EM uses soft cluster assignment: for each instance during the expectation step, the algorithm estimates the probabilty that it belongs to each cluster (based on the current cluster parameters).

### Anomaly Detection using Gaussian Mixtures
*Anomaly dection* (also called *outlier detection*) is the task of detecting instances that deviate strongl from the norm. <br>
Using a Gaussian mixture model for anomaly detecting: any instance located in a low-density region can be considered an anomaly. Density threshold must be defined.

In [ ]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities < density_threshold]

*Novelty detection* differs from anomaly detection in that the algorithm is assumed to be trained on a "clean" dataset, uncontaminated by outliers, whereas anomaly detection does not make this assumption.

### Selecting the Number of Clusters
Find the model that minimizes a *theoretical information criterion* such as the *Bayesian information criterion* (BIC) or the *Akaike information criterion* (AIC). 

$BIC = log(m)p - 2 long(\hat{L})$ <br>
$AIC = 2p - 2 log(\hat{L})$ <br>
- $m$ is the number of instances
- $p$ is the number of parameters learned by the model.
- $\hat{L}$ is the maximized value of the likelihood function of the model.

Both the BIC and the AIC penalize models that have more parameters to learn (e.g., more clusters), and reward models that fit the data well.

Given a statistical model with some parameters $\theta$, the word "probability" is used to describe how plausible a future outcome x is (knowing the parameter values $\theta$), while the word "likelihood" is used to describe how plausible a particular set of parameter values $\theta$ are, after the outcome x is known. <br>
Given a dataset X, a common task is to try to estimate the most likely values for the model parameters. To do this, you must find the values that maximize the likelihood function, given X.


In [ ]:
gm.bic(X)
# 8189.74345832983
gm.aic(X)
# 8102.518178214792

### Bayesian Gaussian Mixture Models
The `BayesianGaussianMixture` class is capable of giving weights equal (or close) to zero to unnecessary clusters.

In [ ]:
from sklearn.mixture import BayesianGaussianMixture

bgm = BayesianGaussianMixture(n_components=10, n_init=10, random_state=42)
bmg.fit(X)
np.round(bgm.weights_, 2)
# array([0.4 , 0.21, 0.4 , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

Gaussian mixture models work great on clusters with ellipsoidal shapes, they may perform poorly with different shapes.

### Other Anomaly Detection and Novelty Detection Algorithms
- *Fast-MCD* (minimum covariance determinant): this algorithm is useful for outlier detection, in particular to cleanup a dataset. It assumes that the normal instances are generated from a single Gaussian distribution, but it also assumes that the dataset is contaminated with outliers that were not generated from this Gaussian distribution.
-  *Isolation forest*: this is an efficient algorithm for outlier detection, especially in high-dimensional datasets. The algorithm builds a Random Forest in which each Decision Tree is grown randomly.
- *Local outlier factor* (LOF): this algorithm is good for outlier detection. It compares the density of instances around a given instance to the density around its neighbors.
- *One-class SVM*: this algorithm is better suited for novelty detection.